# **PART 1**: Multi-fidelity Bayesian optimization (MFBO) with F3DASM

This Jupyter notebook contains the workflow to obtain the results presented in the mulfi-fidelity review manuscript.

## 0. Clone the correct version of F3DASM.
- Follow the procedure described on https://bessagroup.github.io/F3DASM/gettingstarted.html to access the `f3dasm` package.
- Clone F3DASM and select the `versionleo` branch: https://github.com/bessagroup/F3DASM/tree/versionleo

### 1. Import the necessary packages.

In [ ]:
import f3dasm
import numpy as np

### 2. Define the hyperparameters

Some hyperparameters are entered by default, for example the Gaussian process kernel and the multi-fidelity acquisition function.

In [ ]:
dim = 1
iterations = 10
seed = 123
samp_nos = [20, 10]
fids = [0.5, 1.]
costs = [0.5, 1.]

### 3. Specify the multi-fidelity problem

- This means that the functions or models that represent the different fidelities need to be defined and collected into a multi-fidelity function. 

- The design space corresponding to this multi-fidelity function is also needed, in order to define a sampler to sample initial training data from.

The review manuscript works with augmented analytical functions to create the basis of a multi-fidelity problem. 

This starts by picking a base function.

In [ ]:
base_fun = f3dasm.functions.AlpineN2(
    dimensionality=dim,
    scale_bounds=np.tile([0.0, 1.0], (dim, 1)),
    )

Once the function has been defined, the pipeline function `create_analytical_mf_problem` can be used to generate the multi-fidelity assets needed to run the optimization problem.

In [ ]:

mf_train_data, mffun, _, mf_sampler = f3dasm.create_analytical_mf_problem(
    base_fun=base_fun,
    dim=dim,
    fids=fids,
    costs=costs,
    samp_nos=samp_nos,
    )

### 4. Define and initialize optimizer

This optimizer makes use of the variable-fidelity UCB acquisition function.

```
Jiang, P.; Cheng, J.; Zhou, Q.; Shu, L. & Hu, J.
Variable-fidelity lower confidence bounding approach for engineering optimization problems with expensive simulations
AIAA Journal, American Institute of Aeronautics and Astronautics, 2019, 57, 5416-5430
```

In [ ]:
optimizer = f3dasm.optimization.MFBayesianOptimizationTorch(
    data=mf_train_data,
    mffun=mffun,
)

optimizer.init_parameters()

### 5. Run optimization and obtain results

In [ ]:

res = f3dasm.run_mf_optimization(
    optimizer=optimizer,
    mffunction=mffun,
    sampler=mf_sampler[-1],
    iterations=iterations,
    seed=123,
    number_of_samples=samp_nos,
)


In [ ]:

res[1].data